
# Data labeling  
En este notebook lo que haremos es por un lado tomar el dataset que contiene las 10000 laptops descargadas y comprender las columnas que obtuvimos.
Por otro lado lo que haremos es cargar en una columna nueva un path a donde se encuentran ubicadas las imagenes de este laptop. 
Una vez que entendamos cuantas laptops con fotos tenemos y cuantas pudimos etiquetar, separaremos el dataset en 3. Una parte para entrenamiento, otra para validacion y otra para test. Aquellas que quedaron sin etiqueta son las que querremos etiquetar para poder incorporarlas a nuestra database de donde sacaremos las recomendaciones a los clientes.

El etiquetado buscara clasificar las notebooks en 3 tipos:

- Tradicional economica
- Tradicional premium
- Gamers

In [1]:
import pandas as pd
import os
import json
import ast
import numpy as np
from pyprojroot import here

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
path = here() / "data"

In [4]:
file = input("Ingrese el nombre del file que desea leer: \n")

In [5]:
df = pd.read_csv(path / file,  sep = ";")

In [6]:
df.head()

,id,title,price,available_quantity,shipping_cost,date_created,cantidad_de_nucleos,tarjeta_grafica,tipo_tarjeta_grafica,es_2_en_1,es_gamer,es_ultrabook,marca_del_procesador,con_pantalla_tactil,marca,linea_procesador,capacidad_ram,valor_peso,valor_screen,tipo_tarjeta_grafica.1,capacidad_ssd,permalink,ratio_sold
0,MLA1157214207,"Notebook Exo Smart T33 Gris 14 , Intel Celeron...",63071.00000,50,free,2022-08-24T19:11:55.000Z,2.00000,intel uhd graphics 600,integrada,No,No,No,Intel,No,EXO,Celeron,4.00000,1.35000,14.00000,integrada,64.00000,https://articulo.mercadolibre.com.ar/MLA-11572...,0.09091
1,MLA930347589,Notebook Lenovo Ideapad 14iil05 Platinum Gray...,129999.00000,1,free,2021-07-21T18:17:49.000Z,4.00000,intel uhd graphics g1,integrada,No,Sí,Sí,Intel,No,Lenovo,Core i5,8.00000,1.60000,14.00000,integrada,512.00000,https://articulo.mercadolibre.com.ar/MLA-93034...,0.99502
2,MLA1138721902,"Notebook Hp 14-dq2029la Plata Natural 14 , Int...",151999.00000,1,free,2022-05-24T18:58:45.000Z,4.00000,intel iris xe graphics g7 80eus,integrada,No,No,No,Intel,No,HP,Core i5,8.00000,1.46000,14.00000,integrada,256.00000,https://articulo.mercadolibre.com.ar/MLA-11387...,0.99602
3,MLA1144765954,"Notebook Hp 14-cf2531la Negra 14 , Intel Celer...",79999.00000,250,free,2022-06-29T12:31:09.000Z,2.00000,intel uhd graphics 600,integrada,No,No,No,Intel,No,HP,Celeron,4.00000,1.47000,14.00000,integrada,128.00000,https://articulo.mercadolibre.com.ar/MLA-11447...,0.16667
4,MLA1151530200,Notebook Lenovo Ideapad 3 14iil05 Intel Core I...,94999.00000,50,free,2022-08-11T16:12:19.000Z,2.00000,intel uhd graphics,integrada,No,No,No,Intel,No,Lenovo,Core i3,8.00000,1.60000,14.00000,integrada,128.00000,https://articulo.mercadolibre.com.ar/MLA-11515...,0.75000


In [7]:
df.shape

(5721, 23)

### Cargamos el dataset con todas las combinaciones de respuestas posibles y una notebook etiquetada

In [8]:
respuestas = pd.read_csv(path / "respuestas_posibles.csv",  sep = ",",  encoding='windows-1251')

In [9]:
respuestas.shape

(486, 6)

In [10]:
respuestas.head()

,gasto,traslado,uso,teclado numerico,cuotas,comprado
0,$141.000 - $180.000,no,cotidiano,no,si,MLA1142017972
1,$141.000 - $180.000,no,cotidiano,no,no,MLA1132785865
2,$141.000 - $180.000,no,cotidiano,no,tal vez,MLA873012949
3,$141.000 - $180.000,no,cotidiano,si,si,MLA1149165823
4,$141.000 - $180.000,no,cotidiano,si,no,MLA917107922


In [11]:
del respuestas["cuotas"]

In [12]:
respuestas.shape

(486, 5)

In [13]:
respuestas.groupby(["gasto", "traslado", "uso", "teclado numerico"]).size().reset_index(name = "q").sort_values(by = "q", ascending= False).head()

,gasto,traslado,uso,teclado numerico,q
0,$141.000 - $180.000,no,cotidiano,no,3
121,Hasta $140.000,si,disenio,si,3
103,$261.000 - $300.000,tal vez,disenio,si,3
104,$261.000 - $300.000,tal vez,disenio,tal vez,3
105,$261.000 - $300.000,tal vez,jugar,no,3


In [14]:
re = respuestas.merge(respuestas, on = ["gasto", "traslado", "uso", "teclado numerico"]).merge(respuestas, on = ["gasto", "traslado", "uso", "teclado numerico"])
re.drop_duplicates(inplace= True)

In [15]:
re.head()

,gasto,traslado,uso,teclado numerico,comprado_x,comprado_y,comprado
0,$141.000 - $180.000,no,cotidiano,no,MLA1142017972,MLA1142017972,MLA1142017972
1,$141.000 - $180.000,no,cotidiano,no,MLA1142017972,MLA1142017972,MLA1132785865
2,$141.000 - $180.000,no,cotidiano,no,MLA1142017972,MLA1142017972,MLA873012949
3,$141.000 - $180.000,no,cotidiano,no,MLA1142017972,MLA1132785865,MLA1142017972
4,$141.000 - $180.000,no,cotidiano,no,MLA1142017972,MLA1132785865,MLA1132785865


### Importamos las distancias de Gower entre las PCs

In [16]:
file2 = input("Ingrese el archivo que tiene los valores de Gower que quiere leer: \n")

In [17]:
df_gower = pd.read_csv(path / file2,  sep = ";")

In [18]:
df_gower.head()

0       1       2       3       4       5       6       7       8  \
0 0.00000 0.44794 0.35453 0.22389 0.34013 0.37036 0.44698 0.24651 0.34975   
1 0.44794 0.00000 0.35509 0.44179 0.36867 0.48121 0.43771 0.45365 0.40253   
2 0.35453 0.35509 0.00000 0.30514 0.32213 0.43128 0.26937 0.36024 0.26588   
3 0.22389 0.44179 0.30514 0.00000 0.33399 0.36998 0.39735 0.24325 0.30013   
4 0.34013 0.36867 0.32213 0.33399 0.00000 0.39192 0.41216 0.34947 0.27810   

        9      10      11      12      13      14      15      16      17  \
0 0.35933 0.35800 0.34377 0.20263 0.24043 0.22861 0.38787 0.39188 0.36009   
1 0.35693 0.36415 0.40852 0.41923 0.42629 0.43672 0.32251 0.36572 0.39515   
2 0.22325 0.18306 0.31511 0.32581 0.33288 0.34331 0.28089 0.35854 0.26787   
3 0.35318 0.31128 0.33956 0.24600 0.23733 0.22719 0.38342 0.38864 0.31191   
4 0.32355 0.32827 0.26959 0.31142 0.31849 0.32891 0.36420 0.36215 0.32890   

       18      19      20      21      22      23      24      25      26  \
0 0.43493 0.22070 0.39650 0.38505 0.36114 0.22260 0.35897 0.38828 0.38669   
1 0.49160 0.40570 0.44987 0.41071 0.35532 0.45050 0.30767 0.40894 0.40907   
2 0.35584 0.35577 0.36607 0.36417 0.22804 0.35709 0.26832 0.36240 0.35914   
3 0.38530 0.22509 0.39326 0.37891 0.35790 0.22610 0.35282 0.38213 0.38482   
4 0.35760 0.29790 0.36677 0.31025 0.33142 0.34270 0.23865 0.27016 0.31978   

       27      28      29      30      31      32      33      34      35  \
0 0.25029 0.35369 0.35144 0.31490 0.34061 0.40478 0.29424 0.34767 0.21327   
1 0.41513 0.35528 0.31922 0.39408 0.37304 0.44159 0.41457 0.40514 0.40877   
2 0.32172 0.21847 0.27196 0.25727 0.36659 0.35778 0.32116 0.31261 0.31535   
3 0.29302 0.30406 0.34529 0.27187 0.33760 0.40154 0.29237 0.34465 0.25406   
4 0.31459 0.27757 0.22869 0.29621 0.31606 0.37505 0.31411 0.32239 0.31948   

       36      37      38      39      40      41      42      43      44  \
0 0.32667 0.38605 0.23599 0.34445 0.46049 0.38890 0.34454 0.52262 0.39218   
1 0.39229 0.44681 0.42969 0.40951 0.40609 0.45034 0.37727 0.49646 0.45177   
2 0.26840 0.39688 0.29304 0.31564 0.48965 0.31369 0.37081 0.48928 0.31415   
3 0.28364 0.38594 0.18601 0.34121 0.46060 0.33928 0.34182 0.51938 0.34425   
4 0.34831 0.34271 0.32188 0.22776 0.48213 0.31757 0.36377 0.49289 0.31776   

       45      46      47      48      49      50      51      52      53  \
0 0.25195 0.30356 0.44091 0.27667 0.34528 0.27450 0.44395 0.49445 0.39590   
1 0.41459 0.40525 0.18097 0.42271 0.36415 0.45076 0.44846 0.53213 0.31593   
2 0.32118 0.26957 0.36212 0.32929 0.22688 0.35831 0.40192 0.40379 0.27560   
3 0.24871 0.25400 0.43476 0.27343 0.29565 0.27502 0.43780 0.44482 0.34918   
4 0.31413 0.31845 0.36165 0.33868 0.27362 0.29947 0.36469 0.46181 0.36617   

       54      55      56      57      58      59      60      61      62  \
0 0.35302 0.24085 0.37889 0.36202 0.40086 0.41366 0.39178 0.34685 0.34456   
1 0.35651 0.42448 0.46278 0.39700 0.36364 0.31280 0.36561 0.36481 0.37725   
2 0.17542 0.28856 0.37276 0.39079 0.32331 0.40162 0.35844 0.27068 0.37080   
3 0.30364 0.19132 0.37274 0.36239 0.35341 0.41668 0.38853 0.34361 0.34181   
4 0.32063 0.32128 0.36489 0.38609 0.37040 0.44136 0.36205 0.31712 0.36375   

       63      64      65      66      67      68      69      70      71  \
0 0.53039 0.39688 0.34012 0.27579 0.39258 0.40333 0.35002 0.20764 0.38868   
1 0.49020 0.36039 0.45565 0.45068 0.40845 0.44903 0.36128 0.41422 0.40829   
2 0.48302 0.32006 0.36320 0.35726 0.35779 0.36732 0.27119 0.32080 0.36175   
3 0.52593 0.35016 0.33397 0.27880 0.38789 0.39718 0.34387 0.24834 0.38254   
4 0.49944 0.36715 0.31171 0.34739 0.31792 0.36754 0.31423 0.31376 0.31388   

       72      73      74      75      76      77      78      79      80  \
0 0.19916 0.39822 0.32508 0.38876 0.43908 0.37070 0.34892 0.39521 0.36265   
1 0.42293 0.45296 0.39523 0.40943 0.48712 0.49021 0.37359 0.44995 0.48410   
2 0.32952 0.36235 0.26690 0.36288 0.43719 0.39947 0.19250 0.31573 0.34810

In [19]:
len(df.id.to_list())

5721

In [20]:
df_gower.columns = df.id.to_list()
df_gower.index = df.id.to_list()

In [23]:
dic = dict()
for col in df_gower.columns:
   # dic[col] = df_gower[col].sort_values().head(4).values
    dic[col] = df_gower[col].sort_values().head(11).index

    

In [24]:
dic

{'MLA1157214207': Index(['MLA1157214207', 'MLA1148063329', 'MLA1104393499', 'MLA1146916800',
        'MLA1150777628', 'MLA1153325085', 'MLA1149290314', 'MLA1149789231',
        'MLA1149412849', 'MLA1150674708', 'MLA1149278461'],
       dtype='object'),
 'MLA930347589': Index(['MLA930347589', 'MLA912135674', 'MLA910590123', 'MLA1119127006',
        'MLA910730395', 'MLA1134680053', 'MLA1134693002', 'MLA1119114235',
        'MLA907732908', 'MLA910525645', 'MLA926783690'],
       dtype='object'),
 'MLA1138721902': Index(['MLA1138721902', 'MLA1138747432', 'MLA1116611992', 'MLA1114388651',
        'MLA1100473869', 'MLA1134744389', 'MLA1130529922', 'MLA1123071379',
        'MLA1138734538', 'MLA787076937', 'MLA1147642680'],
       dtype='object'),
 'MLA1144765954': Index(['MLA1144765954', 'MLA1136593790', 'MLA1146456346', 'MLA1119160204',
        'MLA1150068687', 'MLA1141901191', 'MLA1157351116', 'MLA1157592752',
        'MLA1148497198', 'MLA1152588436', 'MLA1142228721'],
       dtype='object'

### Facilidades de Pago

In [25]:
file3 = input("Ingrese el nombre del archivo que tiene la infromación de las notebooks que tienen cuotas: \n")

In [27]:
df_facilidades = pd.read_csv(path / file3, sep = ";")

In [28]:
df_facilidades.head()

,id,installment
0,MLA1100042282,yes
1,MLA1150767163,yes
2,MLA1150788397,yes
3,MLA1150783332,yes
4,MLA1150783332,no_interest



## Recommendation